## Create Custom Dataset

In [1]:
# Custom Pytorch dataset
# Class Dataset, Dataloader
# Dataset stores all information regarding dataset (samples, annotations ...)
# Dataloader is an iterator wrapper around dataset, that allows to load different samples in real time

from torch.utils.data import Dataset, DataLoader
import pandas as pd
import torchaudio
import os

#Our UrbanSoundDataset class inherits from the base Dataset class
class UrbanSoundDataset(Dataset):
    
    def __init__(self, annotations_file, audio_dir):
        self.annotations = pd.read_csv(annotations_file)
        self.audio_dir = audio_dir
        
    # Len is method we use to define how we use the len syntax.
    # len(usd) will return the length of the dataset (number of samples).
    def __len__(self):
        return len(self.annotations)
    
    # getitem is a method we use to define how to get a sample from the dataset.
    # work like : a_list[1] -> a_list.__getitem__(1)
    # we want to define functions, that will specify, how to get data from dataset.
    def __getitem__(self, index):
        # Loading waveform of audio sample associated with certain inddex and also the label
        # Path to the sample:
        # self._get_audio_sample - private methods
        audio_sample_path = self._get_audio_sample_path(index)
        label = self._get_audio_sample_label(index)
        # Load audio file using torch audio
        signal, sr = torchaudio.load(audio_sample_path)
        return signal, label
    
    def _get_audio_sample_path(self, index):
        fold = f"fold{self.annotations.iloc[index, 5]}"
        path = os.path.join(self.audio_dir, fold, self.annotations.iloc[index, 0])
        return path
        
    def _get_audio_sample_label(self, index):
        return self.annotations.iloc[index, 6]

        

d:\Code\ProjectsPython\ML_TrainingGround\ML_Audio\.venv\Lib\site-packages\torchaudio\backend\utils.py:74: UserWarning: No audio backend is available.
  warnings.warn("No audio backend is available.")


In [2]:
if __name__ == "__main__":
    ANNOTATIONS_FILE = "data/UrbanSound8K/metadata/UrbanSound8K.csv"
    AUDIO_DIR = "data/UrbanSound8K/audio"
    
    usd = UrbanSoundDataset(ANNOTATIONS_FILE, AUDIO_DIR)
    
    print(f"There are {len(usd)} samples in the dataset.")
    
    signal, label = usd[0]
    
    a = 1

There are 8732 samples in the dataset.


RuntimeError: No audio I/O backend is available.